### Background

In this guide, we'll explore how to use Amazon Aurora PostgreSQL as a vector store for performing similarity search. Vector stores are essential for various natural language processing (NLP) tasks, such as semantic search, recommendation systems, and question-answering.

Amazon Aurora PostgreSQL is a fully managed, PostgreSQL-compatible relational database service provided by Amazon Web Services (AWS). It offers high performance, scalability, and compatibility with PostgreSQL, making it a suitable choice for storing and querying vector data.

We'll leverage a pre-trained model *amazon.titan-embed-g1-text-02*, provided by [Amazon Bedrock](https://docs.aws.amazon.com/bedrock/latest/userguide/titan-embedding-models.html), to create vector embeddings offline and store them for subsequent retrieval operations.

In [ ]:
!pip install -U psycopg2-binary pgvector tqdm boto3 requests numpy==1.23 pandas bottleneck pandarallel

## Load and preprocess data 

Downlad a JSON file containing product metadata from a GitHub repository. The data includes high-resolution fashion images and textual descriptions in English. The data is loaded into a pandas DataFrame for further processing. 

In [ ]:
import urllib.request
import os
import json
import pandas as pd

filename = 'metadata.json'

def download_metadata(url):
    if not os.path.exists(filename):
        urllib.request.urlretrieve(url, filename)

# It is multi-modal Zalando research FEIDEGGER data with 8,732 high-resolution fashion images and five textual annotations in German that has been translated to english using Amazon translate. 
download_metadata('https://raw.githubusercontent.com/aws-samples/rds-postgresql-pgvector/master/data/FEIDEGGER_release_1.2.json')

with open(filename) as json_file:
    data = json.load(json_file)
    df = pd.json_normalize(data)

In [ ]:
# Initialize bedrock client 
import boto3
import json

bedrock = boto3.client(service_name="bedrock")
bedrock_runtime = boto3.client(service_name="bedrock-runtime")

## Generate Text Embeddings

Define a function to generate text embeddings using the Amazon Titan Text Embeddings model. 

In [ ]:
def generate_embeddings(query):
    
    payLoad = json.dumps({'inputText': query })
    response = bedrock_runtime.invoke_model(
        body=payLoad, 
        modelId='amazon.titan-embed-g1-text-02',
        accept="application/json", 
        contentType="application/json" )
    response_body = json.loads(response.get("body").read())
    return(response_body.get("embedding"))

joined_description = ' '.join(df.iloc[1].get('descriptions'))

description_embeddings = generate_embeddings(joined_description)

# Get number of dimensions of the vector embeddings. 
print ("Number of dimensions : {}".format(len(description_embeddings)))


In [ ]:
# Please re-run this cell if it fails the first time with SSL error
from pandarallel import pandarallel as pdp

pdp.initialize(nb_workers=8)

def join_descriptions(descriptions):
    return ' '.join(descriptions)

# Uncomment following line to read first 100 items and reduce the execution time
df = df.head(100)
    
# Join the list of descriptions for each product to match inputText type
df['joined_descriptions'] = df['descriptions'].parallel_apply(join_descriptions)

pdp.initialize(progress_bar=True,nb_workers=8)
# Generate Embeddings for each product
df['descriptions_embeddings'] = df['joined_descriptions'].parallel_apply(generate_embeddings)

print("Completed generation of embeddings for all the products descriptions")

## Store and Index Text embeddings in Aurora PostgreSQL using pgvector extension 

pgvector is an open-source extension for the PostgreSQL that enables efficient storage and search of high-dimensional vector data, such as text embeddings, image features, or any other numerical vector representations.

The primary purpose of pgvector is to provide a seamless integration of vector operations within PostgreSQL, allowing users to perform various similarity searches and nearest neighbor queries on vector data. It introduces new data types, functions, and indexing methods specifically designed for working with vector data

In this step, you will establish a connection to a PostgreSQL database hosted on Amazon Aurora and create a table to store the product data, including the generated text embeddings. 

To enable efficient similarity search on the text embeddings, we'll create an *IVFFlat* short for Inverted File with Flat Compression index on the descriptions_embeddings column with 100 lists (cluster). The ivfflat index is specifically designed for high-dimensional vector data and allows for fast nearest neighbor searches based on vector distances.

In [ ]:
import psycopg2
from pgvector.psycopg2 import register_vector
import boto3 
import json 

client = boto3.client('secretsmanager')

# Replace SecretId with your own 

response = client.get_secret_value(
    SecretId='input-your-secret-id'
)
database_secrets = json.loads(response['SecretString'])

dbhost = database_secrets['host']
dbport = '5432'
dbuser = database_secrets['username']
dbpass = database_secrets['password']

dbconn = psycopg2.connect(host=dbhost, user=dbuser, password=dbpass, port=dbport, connect_timeout=10)
dbconn.set_session(autocommit=True)

print ("RDS connection successful")

cur = dbconn.cursor()

cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
register_vector(dbconn)
cur.execute("DROP TABLE IF EXISTS products;")
cur.execute("""CREATE TABLE IF NOT EXISTS products(
               id bigserial primary key, 
               description text, 
               url text, 
               split int, 
               descriptions_embeddings vector(1536));""")


for _, x in df.iterrows():
    cur.execute("""INSERT INTO products
                      (description, url, split, descriptions_embeddings) 
                   VALUES(%s, %s, %s, %s);""", 
                   (' '.join(x.get('descriptions', [])), x.get('url'), x.get('split'), x.get('descriptions_embeddings') ))

# Indexing the vector embeddings using ivfflat indexing method
cur.execute("""CREATE INDEX ON products 
               USING ivfflat (descriptions_embeddings vector_l2_ops) WITH (lists = 100);""")
cur.execute("VACUUM ANALYZE products;")

cur.close()
dbconn.close()

print ("Vector embeddings has been successfully loaded into PostgreSQL")

## Evaluate PostgreSQL vector Search Results

We will generate text embeddings for the query using the Amazon Titan Text Embeddings model. These embeddings will be used to search a PostgreSQL database for the nearest neighbor product descriptions based on vector similarity. 

In [ ]:
import numpy as np
import requests
from IPython.display import display, Markdown, Latex, HTML

import matplotlib.pyplot as plt
from skimage import io


def similarity_search(search_text):
    
    embedding = np.array(generate_embeddings(search_text))
    
    dbconn = psycopg2.connect(host=dbhost, user=dbuser, password=dbpass, port=dbport, connect_timeout=10)
    dbconn.set_session(autocommit=True)
    cur = dbconn.cursor()

    try:
        cur.execute("""SELECT id, url, description, descriptions_embeddings 
                FROM products 
                ORDER BY descriptions_embeddings <-> %s limit 2;""",(embedding,))
        r = cur.fetchall()
    except Exception as e:
        print(f"Error executing SQL query: {e}")                

    plt.rcParams["figure.figsize"] = [7.50, 3.50]
    plt.rcParams["figure.autolayout"] = True
    urls = []
  
    for x in r:
        url = x[1].split('?')[0]
        urldata = requests.get(url).content
        print ("Product Item Id: " + str(x[0]))
        a = io.imread(url)
        plt.imshow(a)
        plt.axis('off')
        plt.show()

    cur.close()
    dbconn.close()

print("Created similarity_search function successfully")


In [ ]:
similarity_search("purple flower dress")